In [1]:
import json
import spacy
import yaml
import os
import operator
import copy
from spacy.vocab import Vocab
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
def import_bios(path):
    ids = []
    bio = []
    json_files = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
    for i in json_files:
        with open(path+i) as fp:  
            temp = json.load(fp)
            i = [i['_id'] for i in temp['results'] if i.get('bio')]
            b = [i.get('bio') for i in temp['results'] if i.get('bio')]
        ids+=i;
        bio+=b
    return ids,bio

In [4]:
# with open('../../../config/tinder.yaml', 'r') as ymlfile:
#         cfg = yaml.load(ymlfile)
#         path = cfg['output_folder']+'tinder/'+cfg['city']+'/'
#         json_files = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
#         data_path = ('../../../output/data')
#         if not os.path.exists(data_path):
#                 os.mkdir(data_path)
path = '/home/pallavi/data/'
ids,bios = import_bios(path)
df = pd.DataFrame(list(zip(ids, bios)), columns =['ID', 'Bio']) 
df.head()

,ID,Bio
0,59659967fa14850173b0a87c,well i took 3 different buzzfeed quizzes and t...
1,5ac27b2fe3fdb7f9234fd14b,if you can hold a conversation you can have my...
2,5ccfb8205a93401500a51dc3,The one with the freckle
3,5ce0915013775b1f004b6f83,5’2” if it matters
4,5cac0583f8db7516001eaf48,Looks are a generous 7/10 but personality is a...


In [5]:
len(df)

2070

In [6]:
def lemma_sw_dict(bios):
    nlp = spacy.load('en')
    words = set(nltk.corpus.words.words())
    stop_words = set(stopwords.words('english'))
    clean = bios
    for i, doc in enumerate(clean):
        doc = nlp(doc)
        clean[i] = " ".join([token.lemma_ for token in doc if (str(token.lemma_) in words and str(token.lemma_) not in stop_words)])
    df['cleaned_bio'] = clean

In [7]:
lemma_sw_dict(bios)
df.head()

,ID,Bio,cleaned_bio
0,59659967fa14850173b0a87c,well i took 3 different buzzfeed quizzes and t...,well take different quiz say go find love year...
1,5ac27b2fe3fdb7f9234fd14b,if you can hold a conversation you can have my...,hold conversation snap
2,5ccfb8205a93401500a51dc3,The one with the freckle,one freckle
3,5ce0915013775b1f004b6f83,5’2” if it matters,matter
4,5cac0583f8db7516001eaf48,Looks are a generous 7/10 but personality is a...,look generous personality solid sure


In [8]:
def tfidf(bios):
    vectorizer = TfidfVectorizer(smooth_idf= True)
    X = vectorizer.fit_transform(bios)
    feature_names = vectorizer.get_feature_names()
    vec = X.toarray()
    return vec,feature_names

In [9]:
vector,features=tfidf(bios)

In [10]:
def tf_idf_score(vec):
    score = []
    for i in vec:
        score.append(sum(i)/len(i))
    return score

In [11]:
df['scores'] = tf_idf_score(vector)

In [12]:
df = df.sort_values('scores',ascending=False)
df.head()

,ID,Bio,cleaned_bio,scores
1551,5cc1565caa97e51500ace05b,"Truths about me, if you care\nIf we matched, I...",truth care match likely message snap often bad...,0.001916
39,5988d759b33bbdbd4ba33d12,Boston native\n\nLooking for witty banter and ...,Boston native look witty banter black coffee g...,0.001889
367,5cbdfaa4654fa015008a0cf2,Love practicing hot yoga and testing new recip...,love practice hot yoga test new recipe nutriti...,0.001875
1740,55fdc1bb4a5adedc5624892e,You know what they say about guys with big soc...,know say guy big sock even big heart definitel...,0.001851
1598,5c6374a22512fa110071c370,Jewish boy from New York\nUniversity of Washin...,boy Washington animal nature food booze live m...,0.001815


In [15]:
df.to_csv('/home/pallavi/Project/SmartDate/output/tf-idf-scores.csv', sep='\t',header=True, index=False)